<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/neuro_evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [0]:
import numpy as np
import math
import pandas as pd
import pickle
from numba import jit as njit
from functools import lru_cache
import time
import random
import ta
from net import *
from memory import *
from reward import *

In [0]:
class neuralnetwork:
    def __init__(self, id_, hidden_size = 128*2, window_size = 100*2, restore=False):
        if restore:
          f = open("./neuro_w.txt","rb")
          weights = pickle.load(f)
          self.W1 = weights[0]
          self.W2 = weights[1]
        else:
          self.W1 = np.random.randn(window_size, hidden_size) / np.sqrt(window_size)
          self.W2 = np.random.randn(hidden_size, 3) / np.sqrt(hidden_size)
        self.fitness = 0
        self.id = id_

@njit
def sigmoid(x):
  x = 1 / (1 + np.exp(-x))
  return x

def swish(x):
  x *= sigmoid(x)
  return x
 
def relu(X):
    return np.maximum(X, 0)
  
@njit
def softmax(X):
    e_x = np.exp(X - np.max(X, axis=-1, keepdims=True))
    e_x /= np.sum(e_x, axis=-1, keepdims=True)
    return e_x
  
@njit
def feed_forward(X, nets):
    X = X.flatten().reshape(1,-1)
    a1 = np.dot(X, nets.W1)
    z1 = swish(a1)
    a2 = np.dot(z1, nets.W2)
    return softmax(a2)

In [0]:
class NeuroEvolution:
    def __init__(self, population_size, mutation_rate, model_generator, state_size, window_size, path, step_size, spread=10, pip_cost=1000, los_cut=20,restore=False):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.model_generator = model_generator
        self.state_size = state_size
        self.window_size = window_size
        self.path = path
        self.step_size = step_size
        self.spread = spread / pip_cost
        self.pip_cost = pip_cost
        self.los_cut = los_cut
        self.restore = restore
        self.preproc()
        self.rewards = reward3
        
    def preproc(self):
          self.dat = df = pd.read_csv(self.path)
          s = np.asanyarray(ta.stoch(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1)) - np.asanyarray(ta.stoch_signal(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1))
          x = np.asanyarray(ta.daily_log_return(df["Close"])).reshape((-1,1))
          m = np.asanyarray(ta.macd_diff(df["Close"])).reshape((-1,1))
          x = m
#           x = np.concatenate([x,m], 1)
          y = np.asanyarray(self.dat[["Open"]])

          gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x, y, self.window_size)
          self.x = []
          self.y = []
          for i in gen:
              self.x.extend(i[0].tolist())
              self.y.extend(i[1].tolist())
          self.x = np.asanyarray(self.x)
          self.y = np.asanyarray(self.y)

          self.df = self.x[-self.step_size::]
          self.trend = self.y[-self.step_size::]

    def _initialize_population(self):
        self.population = []
        for i in range(self.population_size):
            self.population.append(self.model_generator(i,window_size = self.window_size*self.df.shape[-1], restore=self.restore))
    
    def mutate(self, individual, scale=1.0):
        mutation_mask = np.random.binomial(1, p=self.mutation_rate, size=individual.W1.shape)
        individual.W1 += np.random.normal(loc=0, scale=scale, size=individual.W1.shape) * mutation_mask
        mutation_mask = np.random.binomial(1, p=self.mutation_rate, size=individual.W2.shape)
        individual.W2 += np.random.normal(loc=0, scale=scale, size=individual.W2.shape) * mutation_mask
        return individual
    
    def inherit_weights(self, parent, child):
        child.W1 = parent.W1.copy()
        child.W2 = parent.W2.copy()
        return child
    
    def crossover(self, parent1, parent2):
        child1 = self.model_generator((parent1.id+1)*10)
        child1 = self.inherit_weights(parent1, child1)
        child2 = self.model_generator((parent2.id+1)*10)
        child2 = self.inherit_weights(parent2, child2)
        # first W
        n_neurons = child1.W1.shape[1]
        cutoff = np.random.randint(0, n_neurons)
        child1.W1[:, cutoff:] = parent2.W1[:, cutoff:].copy()
        child2.W1[:, cutoff:] = parent1.W1[:, cutoff:].copy()
        # second W
        n_neurons = child1.W2.shape[1]
        cutoff = np.random.randint(0, n_neurons)
        child1.W2[:, cutoff:] = parent2.W2[:, cutoff:].copy()
        child2.W2[:, cutoff:] = parent1.W2[:, cutoff:].copy()
        return child1, child2
    
    def act(self, p, state):
        logits = feed_forward(state, p)
        return np.argmax(logits, 1)[0]
#         return np.argmax(logits[0])
    
    def test(self, individual, i):
        states = []
        pip = []
        history = []
        h_p = []
        provisional_pip = []
        total_pip = 0.0
        position = 3
        h = self.h
#         h = np.random.randint(100,self.x.shape[0]-(self.step_size*20))
#         self.df = self.x[h+self.step_size:h+self.step_size*2]
#         self.trend = self.y[h+self.step_size:h+self.step_size*2]
        for t in range(0, len(self.trend) - 1):
            action = self.act(individual, self.df[t])
            history.append(action)
            states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,action,
                                                                     position,states,self.pip_cost,self.spread,total_pip,lc=self.los_cut/1000)
#             print(len(provisional_pip))
            h_p.append(position)
        self.pip = np.asanyarray(provisional_pip) * self.pip_cost
        self.pip = [p if p >= -self.los_cut else -self.los_cut for p in self.pip]
        self.total_pip = np.sum(self.pip)
        mean_pip = self.total_pip / (t + 1)
        trade_accuracy = np.mean(np.asanyarray(self.pip) > 0)
        self.trade = trade_accuracy
        mean_pip *= 24
        prob = self.prob(history)
        position_prob = self.prob(h_p)
      
        print("")
        print('action probability = ', prob)
        print("buy = ", position_prob[1], " sell = ", position_prob[-1])
        print('trade accuracy = ', trade_accuracy)
        print('epoch: %d, total rewards: %f, mean rewards: %f' % (i+1, float(self.total_pip), float(mean_pip)))
    
    def calculate_fitness(self):
        change = True
        for i in range(self.population_size):
          states = []
          pip = []
          provisional_pip = []
          total_pip = 0.0
          position = 3
          if change:
            change = False
            self.h = h = np.random.randint(self.x.shape[0]-(self.step_size+1))
            self.df = self.x[h:h+self.step_size]
            self.trend = self.y[h:h+self.step_size]
          for t in range(0, len(self.trend) - 1):
              action = self.act(self.population[i], self.df[t])
              states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,
                                                                       action,position,states,self.pip_cost,self.spread,total_pip,lc=self.los_cut/1000)
          invest = total_pip * self.pip_cost
          trade_accuracy = np.mean(np.asanyarray(provisional_pip) > 0) * 100
          self.population[i].fitness = trade_accuracy
    
    def prob(self,history):
        prob = np.asanyarray(history)
        a = np.mean(prob == 0)
        b = np.mean(prob == 1)
        c = 1 - (a + b)
        prob = [a,b,c]
        return prob
    
    def evolve(self, generations=20, checkpoint= 5):
        self._initialize_population()
        n_winners = int(self.population_size * 0.6)
        n_parents = self.population_size - n_winners
        for epoch in range(generations):
            self.calculate_fitness()
            self.fitnesses = fitnesses = [i.fitness for i in self.population]
            fitnesses = np.array(neural_evolve.fitnesses).flatten()
            self.sort_fitness = sort_fitness = np.argsort(neural_evolve.fitnesses,None)[::-1]
            self.population = [self.population[int(i)] for i in sort_fitness]
            fittest_individual = self.population[0]
            if (epoch+1) % checkpoint == 0:
                self.test(fittest_individual, epoch)
                save = [fittest_individual.W1, fittest_individual.W2]
                f = open('neuro_w.txt', 'wb')
                pickle.dump(save, f)
            
            next_population = [self.population[i] for i in range(n_winners)]
            total_fitness = np.sum([np.abs(i.fitness) for i in self.population])
            parent_probabilities = [np.abs(i.fitness / total_fitness) for i in self.population]
            sort_probabilities = np.argsort(parent_probabilities,None).reshape(-1,1)
            parent_probabilities = [parent_probabilities[int(i)] for i in sort_probabilities]
            parent_probabilities = np.array(parent_probabilities).flatten()
            parent_probabilities = np.nan_to_num(parent_probabilities)
            parents = np.random.choice(self.population, size=n_parents, p=parent_probabilities, replace=False)
            for i in np.arange(0, len(parents), 2):
                child1, child2 = self.crossover(parents[i], parents[i+1])
                next_population += [self.mutate(child1), self.mutate(child2)]
            self.population = next_population


In [0]:
population_size = 100
generations = 100
mutation_rate = 0.1
window_size = 30
step_size = 96
los_cut = 300
restore = False
path = "audpred15.csv"

neural_evolve = NeuroEvolution(population_size, mutation_rate, neuralnetwork,
                              window_size, window_size, path, step_size,restore=restore, los_cut=los_cut)

In [0]:
fittest_nets = neural_evolve.evolve(50000,1)


action probability =  [0.3368421052631579, 0.42105263157894735, 0.24210526315789482]
buy =  0.5789473684210527  sell =  0.42105263157894735
trade accuracy =  0.7096774193548387
epoch: 1, total rewards: 2470.000000, mean rewards: 624.000000

action probability =  [0.15789473684210525, 0.4631578947368421, 0.3789473684210527]
buy =  0.16842105263157894  sell =  0.8315789473684211
trade accuracy =  0.8235294117647058
epoch: 2, total rewards: 1944.000000, mean rewards: 491.115789

action probability =  [0.16842105263157894, 0.30526315789473685, 0.5263157894736842]
buy =  0.16842105263157894  sell =  0.8315789473684211
trade accuracy =  0.875
epoch: 3, total rewards: 4194.000000, mean rewards: 1059.536842

action probability =  [0.031578947368421054, 0.5263157894736842, 0.4421052631578948]
buy =  0.031578947368421054  sell =  0.968421052631579
trade accuracy =  0.9411764705882353
epoch: 4, total rewards: 2282.000000, mean rewards: 576.505263

action probability =  [0.23157894736842105, 0.35

In [0]:
fit = np.array(neural_evolve.fitnesses).flatten()
sort_fitness = np.argsort(neural_evolve.fitnesses,None)[::-1]
print(sort_fitness)
a = [neural_evolve.population[int(i)] for i in sort_fitness]
# print(fit)
[a.fitness for a in a]

In [0]:
[i.fitness for i in neural_evolve.population]